In [1]:
#pip install xgboost
import pandas as pd
import seaborn as sns
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier 

In [2]:
import matplotlib
matplotlib.use('Agg')

In [3]:
data = pd.read_csv('/Users/aneerajbidlan/Downloads/Problem Statement 1/heart_cleveland_upload.csv')

In [4]:
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,40,1,3,152,223,0,0,181,0,0.0,0,0,2,1
293,39,1,3,118,219,0,0,140,0,1.2,1,0,2,1
294,35,1,3,120,198,0,0,130,1,1.6,1,0,2,1
295,35,0,3,138,183,0,0,182,0,1.4,0,0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        297 non-null    int64  
 1   sex        297 non-null    int64  
 2   cp         297 non-null    int64  
 3   trestbps   297 non-null    int64  
 4   chol       297 non-null    int64  
 5   fbs        297 non-null    int64  
 6   restecg    297 non-null    int64  
 7   thalach    297 non-null    int64  
 8   exang      297 non-null    int64  
 9   oldpeak    297 non-null    float64
 10  slope      297 non-null    int64  
 11  ca         297 non-null    int64  
 12  thal       297 non-null    int64  
 13  condition  297 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 32.6 KB


Our aim here is to analyze our data and report our findings through visualizations and the code below allows us to check if we have any missing values in our dataset before going further with the analysis.

In [6]:
data.isnull().sum()

age          0
sex          0
cp           0
trestbps     0
chol         0
fbs          0
restecg      0
thalach      0
exang        0
oldpeak      0
slope        0
ca           0
thal         0
condition    0
dtype: int64

#### creating a copy of dataset so that will not affect our original dataset.


In [7]:
heart_df = data.copy()

In [8]:
heart_df.describe().T ## We Use T to round off the values

,count,mean,std,min,25%,50%,75%,max
age,297.0,54.542088,9.049736,29.0,48.0,56.0,61.0,77.0
sex,297.0,0.676768,0.468500,0.0,0.0,1.0,1.0,1.0
cp,297.0,2.158249,0.964859,0.0,2.0,2.0,3.0,3.0
trestbps,297.0,131.693603,17.762806,94.0,120.0,130.0,140.0,200.0
chol,297.0,247.350168,51.997583,126.0,211.0,243.0,276.0,564.0
fbs,297.0,0.144781,0.352474,0.0,0.0,0.0,0.0,1.0
restecg,297.0,0.996633,0.994914,0.0,0.0,1.0,2.0,2.0
thalach,297.0,149.599327,22.941562,71.0,133.0,153.0,166.0,202.0
exang,297.0,0.326599,0.469761,0.0,0.0,0.0,1.0,1.0
oldpeak,297.0,1.055556,1.166123,0.0,0.0,0.8,1.6,6.2


In [9]:
#Rename Our Condition column to Target column
heart_df = heart_df.rename(columns={'condition':'target'})
print(heart_df.head(1))

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   69    1   0       160   234    1        2      131      0      0.1      1   

   ca  thal  target  
0   1     0       0  


# Exploratory data analysis (EDA) 

In [10]:
#We Will be checking the relationship among the variable
corr = heart_df.corr()
plt.figure(figsize = (15,15))
sns.heatmap(corr, annot = True , fmt = ".1f", linewidths = .7, cmap="Blues")

<Axes: >

In [11]:
#fixing our data in x and y. Here y contains target data and X contains rest all the features.
x= heart_df.drop(columns= 'target')
y= heart_df.target

In [12]:
sns.countplot(y)

/Users/aneerajbidlan/opt/anaconda3/lib/python3.10/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


<Axes: xlabel='target', ylabel='count'>

In [13]:
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       297 non-null    int64  
 1   sex       297 non-null    int64  
 2   cp        297 non-null    int64  
 3   trestbps  297 non-null    int64  
 4   chol      297 non-null    int64  
 5   fbs       297 non-null    int64  
 6   restecg   297 non-null    int64  
 7   thalach   297 non-null    int64  
 8   exang     297 non-null    int64  
 9   oldpeak   297 non-null    float64
 10  slope     297 non-null    int64  
 11  ca        297 non-null    int64  
 12  thal      297 non-null    int64  
 13  target    297 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 32.6 KB


In [14]:
cat_cols= ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal','target']

num_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak','target']




In [15]:
print(cat_cols)
print(num_cols)


['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'target']
['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'target']


In [16]:
fig, ax = plt.subplots()
sns.countplot(ax = ax, data = heart_df, x = heart_df['sex'], hue = "target")
for label in ax.containers:
    ax.bar_label(label)
ax.set(ylabel='Counts', title='sex')

[Text(0, 0.5, 'Counts'), Text(0.5, 1.0, 'sex')]

In [17]:
fig, ax = plt.subplots()
sns.countplot(ax = ax, data = heart_df, x = heart_df['cp'], hue = "target")
for label in ax.containers:
    ax.bar_label(label)
ax.set(ylabel='Counts', title='Chest Pain')

[Text(0, 0.5, 'Counts'), Text(0.5, 1.0, 'Chest Pain')]

In [18]:
fig, ax = plt.subplots()
sns.countplot(ax = ax, data = heart_df, x = heart_df['fbs'], hue = "target")
for label in ax.containers:
    ax.bar_label(label)
ax.set(ylabel='Counts', title='Fasting Blood Sugar')

[Text(0, 0.5, 'Counts'), Text(0.5, 1.0, 'Fasting Blood Sugar')]

In [19]:
fig, ax = plt.subplots()
sns.countplot(ax = ax, data = heart_df, x = heart_df['restecg'], hue = "target")
for label in ax.containers:
    ax.bar_label(label)
ax.set(ylabel='Counts', title='Rest ECG')

[Text(0, 0.5, 'Counts'), Text(0.5, 1.0, 'Rest ECG')]

In [20]:
fig, ax = plt.subplots()
sns.countplot(ax = ax, data = heart_df, x = heart_df['exang'], hue = "target")
for label in ax.containers:
    ax.bar_label(label)
ax.set(ylabel='Counts', title='Exang')

[Text(0, 0.5, 'Counts'), Text(0.5, 1.0, 'Exang')]

In [21]:
fig, ax = plt.subplots()
sns.countplot(ax = ax, data = heart_df, x = heart_df['slope'], hue = "target")
for label in ax.containers:
    ax.bar_label(label)
ax.set(ylabel='Counts', title='slope')

[Text(0, 0.5, 'Counts'), Text(0.5, 1.0, 'slope')]

In [22]:
fig, ax = plt.subplots()
sns.countplot(ax = ax, data = heart_df, x = heart_df['ca'], hue = "target")
for label in ax.containers:
    ax.bar_label(label)
ax.set(ylabel='Counts', title='CA')

[Text(0, 0.5, 'Counts'), Text(0.5, 1.0, 'CA')]

In [23]:
fig, ax = plt.subplots()
sns.countplot(ax = ax, data = heart_df, x = heart_df['thal'], hue = "target")
for label in ax.containers:
    ax.bar_label(label)
ax.set(ylabel='Counts', title='THAL ')

[Text(0, 0.5, 'Counts'), Text(0.5, 1.0, 'THAL ')]

## Numeric Feature Analysis

In [24]:
def histPlot(num):
    sns.histplot(data = heart_df, x = num, bins = 50, kde = True)
    print("{} distribution with hist:".format(num))
    plt.show()

In [25]:
num_cols

['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'target']

In [26]:
sns.histplot(data = heart_df, x = heart_df['age'], bins = 50, kde = True)
#print("{} distribution with hist:".format(heart_df['age']))
plt.show()

/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/1736510940.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [27]:
sns.histplot(data = heart_df, x = heart_df['chol'], bins = 50, kde = True)
#print("{} distribution with hist:".format(heart_df['age']))
plt.show()


/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/2632735381.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [28]:
sns.histplot(data = heart_df, x = heart_df['trestbps'], bins = 50, kde = True)
#print("{} distribution with hist:".format(heart_df['age']))
plt.show()


/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/1321723205.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [29]:
sns.histplot(data = heart_df, x = heart_df['thalach'], bins = 50, kde = True)
#print("{} distribution with hist:".format(heart_df['age']))
plt.show()

/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/920626799.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [30]:
sns.histplot(data = heart_df, x = heart_df['oldpeak'], bins = 50, kde = True)
#print("{} distribution with hist:".format(heart_df['age']))
plt.show()

/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/2581621753.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [31]:
for i in ['trestbps', 'chol', 'thalach']:
    plt.figure(figsize=(12,5))
    sns.lineplot(y=i,x="age",data=heart_df)
    plt.title(f"{i} WITH AGE",fontsize=20)
    plt.xlabel(i,fontsize=15)
    plt.ylabel(i,fontsize=15)
    plt.show()

/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/1315165354.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/1315165354.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/1315165354.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


There is no strong Relationship with age and heart attack.So we can't say with Increasing the Age There is high Chance of Heart attack or Low Chance of Heart Attack.

There is high chance of Increase in Blood Pressure in the body With Increase in Age.

There is high chance of Increase in Cholestrol Level in the body with increase in Age.

There is high chance of Increase in Heart Rate in the body with increase in Age

## Encoding the data

In [32]:
cat_cols= ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

num_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']




In [33]:
#heart_df = pd.get_dummies(heart_df,columns=cat_cols,drop_first=True)


In [34]:
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       297 non-null    int64  
 1   sex       297 non-null    int64  
 2   cp        297 non-null    int64  
 3   trestbps  297 non-null    int64  
 4   chol      297 non-null    int64  
 5   fbs       297 non-null    int64  
 6   restecg   297 non-null    int64  
 7   thalach   297 non-null    int64  
 8   exang     297 non-null    int64  
 9   oldpeak   297 non-null    float64
 10  slope     297 non-null    int64  
 11  ca        297 non-null    int64  
 12  thal      297 non-null    int64  
 13  target    297 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 32.6 KB


In [35]:
x = heart_df.drop(['target'],axis=1)
y = heart_df[['target']]

## Spliting our data set into train and test set

In [36]:
# Spliting our data set into train and test set
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [37]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222 entries, 66 to 102
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       222 non-null    int64  
 1   sex       222 non-null    int64  
 2   cp        222 non-null    int64  
 3   trestbps  222 non-null    int64  
 4   chol      222 non-null    int64  
 5   fbs       222 non-null    int64  
 6   restecg   222 non-null    int64  
 7   thalach   222 non-null    int64  
 8   exang     222 non-null    int64  
 9   oldpeak   222 non-null    float64
 10  slope     222 non-null    int64  
 11  ca        222 non-null    int64  
 12  thal      222 non-null    int64  
dtypes: float64(1), int64(12)
memory usage: 24.3 KB


In [38]:
scaler = StandardScaler()
scaler

StandardScaler()

In [39]:
x_train_scaler=scaler.fit_transform(x_train)
x_test_scaler=scaler.fit_transform(x_test)

## Modeling

### Logistic Regression

In [40]:
model_lr=LogisticRegression()

In [41]:
y_train = np.array(y_train)
y_traintmp = y_train.ravel()
y_train = np.array(y_traintmp).astype(int)
type(y_train)

numpy.ndarray

In [42]:
model_lr.fit(x_train_scaler,y_train)

LogisticRegression()

In [43]:
y_test_pred = model_lr.predict(x_test_scaler)

In [44]:
p=model_lr.score(x_test_scaler,y_test_pred)

In [45]:
#filename = 'Heart_Attack_Prediction_LR.h5'
pickle.dump(model_lr, open('model_lr.pkl', 'wb'))

In [46]:
type(y_test), type(y_test_pred)

(pandas.core.frame.DataFrame, numpy.ndarray)

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(y_test,y_test_pred)
print("Test score: {}".format(accuracy_score))

Test score: 0.76


In [48]:
print(confusion_matrix(y_test, y_test_pred))

[[30  8]
 [10 27]]


In [49]:
y_pred_prob = model_lr.predict_proba(x_test_scaler)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1])

In [50]:
plt.plot([0,1],[0,1],"k--")
plt.plot(fpr, tpr, label = "Logistic Regression")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Logistic Regression ROC Curve")
plt.show()

/var/folders/dv/7tp6s8kx3m5g9n766pyg_n300000gn/T/ipykernel_5618/4119256259.py:6: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Creating K-Nearest-Neighbor classifier

In [51]:
sel=SelectFromModel(RandomForestClassifier())

In [52]:
sel.fit(x_train_scaler,y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [53]:
sel.get_support()

array([ True, False,  True,  True,  True, False, False,  True, False,
        True, False,  True,  True])

In [54]:
selected_feat= x_train.columns[(sel.get_support())]
selected_feat

Index(['age', 'cp', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca', 'thal'], dtype='object')

In [55]:
#y_train = np.array(y_train)

In [56]:
#type(x_train_scaler), type(y_train)

In [57]:
#x_train_scaler = pd.DataFrame(x_train_scaler)
#type(x_train_scaler)

In [58]:
model_RMC=RandomForestClassifier()
model_RMC.fit(x_train_scaler, y_train)
y_test_pred = model_RMC.predict(x_test_scaler)


from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(y_test,y_test_pred)
print("Test score: {}".format(accuracy_score))


Test score: 0.8


In [59]:
#filename = 'Heart_Attack_Prediction_LR.h5'
pickle.dump(model_RMC, open('model_RMC.pkl', 'wb'))

## XGBClassifier

In [60]:
 model_xgb= XGBClassifier()

In [61]:
model_xgb.fit(x_train_scaler,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [62]:
y_test_pred = model_xgb.predict(x_test_scaler)

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(y_test,y_test_pred)
print("Test score: {}".format(accuracy_score))


Test score: 0.7866666666666666


In [64]:
#filename = 'Heart_Attack_Prediction_LR.h5'
pickle.dump(model_xgb, open('model_xgb.pkl', 'wb'))